In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline
import seaborn as sns; sns.set()

# pd.get_option("display.max_columns")
# pd.set_option('display.max_columns', 100)

参考資料：<br>
https://run.unl.pt/bitstream/10362/33864/1/TGI0135.pdf<br>

In [2]:
DATA_DIR = '/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/WDataFiles_Stage2'
STAGE_1 = True

# シード情報特徴量・qualityの作成

In [90]:
regular_results = pd.read_csv(DATA_DIR+'/WRegularSeasonCompactResults.csv')

In [91]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'         
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]
    output = pd.concat([df, dfswap]).sort_index().reset_index(drop=True)
    
    return output

regular_results = prepare_data(regular_results)
# regular_results = regular_results.loc[:, ['Season', 'T1_TeamID', 'T2_TeamID', 'location', 'NumOT']]
# regular_results = regular_results.loc[regular_results['Season'] >= 2015]
regular_results['T1_win'] = 0
regular_results['T2_win'] = 0
regular_results.loc[regular_results['T1_Score'] > regular_results['T2_Score'], 'T1_win'] = 1
regular_results.loc[regular_results['T1_Score'] < regular_results['T2_Score'], 'T2_win'] = 1
regular_results

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_win,T2_win
0,1998,18,3104,91,3202,41,H,0,1,0
1,1998,18,3202,41,3104,91,A,0,0,1
2,1998,18,3163,87,3221,76,H,0,1,0
3,1998,18,3221,76,3163,87,A,0,0,1
4,1998,18,3222,66,3261,59,H,0,1,0
...,...,...,...,...,...,...,...,...,...,...
231473,2021,132,3195,84,3251,62,N,0,1,0
231474,2021,132,3358,45,3372,56,N,0,0,1
231475,2021,132,3372,56,3358,45,N,0,1,0
231476,2021,132,3133,78,3179,70,N,0,1,0


In [92]:
# convert to str, so the model would treat TeamID them as factors
regular_results['T1_TeamID'] = regular_results['T1_TeamID'].astype(str)
regular_results['T2_TeamID'] = regular_results['T2_TeamID'].astype(str)

# make it a binary task
regular_results['win'] = np.where(regular_results['T1_Score']>regular_results['T2_Score'], 1, 0)

def team_quality(season):
    """
    Calculate team quality for each season seperately. 
    Team strength changes from season to season (students playing change!)
    So pooling everything would be bad approach!
    """
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_results.loc[regular_results.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    
    # extracting parameters from glm
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','beta']
    quality['Season'] = season
    # taking exp due to binomial model being used
    quality['quality'] = np.exp(quality['beta'])
    # only interested in glm parameters with T1_, as T2_ should be mirroring T1_ ones
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(np.int64)
    return quality

In [93]:
%%time
team_quality = pd.concat([

                          team_quality(2015),
                          team_quality(2016),
                          team_quality(2017),
                          team_quality(2018),
                          team_quality(2019),
                          team_quality(2020),
                          team_quality(2021)
                          ]).reset_index(drop=True)

team_quality

/Users/sasanoshouta/.pyenv/versions/3.8.0/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:894: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/Users/sasanoshouta/.pyenv/versions/3.8.0/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:894: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))


CPU times: user 8min 6s, sys: 29.4 s, total: 8min 35s
Wall time: 5min 20s


,TeamID,beta,Season,quality
0,3101,5.404463e-15,2015,1.000000e+00
1,3102,-1.708196e+00,2015,1.811924e-01
2,3103,2.500809e+00,2015,1.219235e+01
3,3104,2.482121e+00,2015,1.196662e+01
4,3105,-2.646451e+00,2015,7.090240e-02
...,...,...,...,...
2436,3467,-2.673165e-01,2021,7.654308e-01
2437,3468,-1.384907e+00,2021,2.503472e-01
2438,3469,-3.925919e+01,2021,8.911509e-18
2439,3470,-1.223336e+00,2021,2.942469e-01


In [94]:
# 各シードに配置された時の勝率、パワーランク作成
tourney_results = pd.read_csv(DATA_DIR+'/WNCAATourneyCompactResults.csv')
seeds = pd.read_csv(DATA_DIR+'/WNCAATourneySeeds.csv')

tourney_results = prepare_data(tourney_results)

team_quality_copy = team_quality.copy()

team_quality_T1 = team_quality_copy[['TeamID','Season','quality']]
team_quality_T1.columns = ['T1_TeamID','Season','T1_quality']
team_quality_T2 = team_quality_copy[['TeamID','Season','quality']]
team_quality_T2.columns = ['T2_TeamID','Season','T2_quality']

tourney_results['T1_TeamID'] = tourney_results['T1_TeamID'].astype('Int64')
tourney_results['T2_TeamID'] = tourney_results['T2_TeamID'].astype('Int64')
tourney_results = tourney_results.merge(team_quality_T1, on = ['T1_TeamID','Season'], how = 'left')
tourney_results = tourney_results.merge(team_quality_T2, on = ['T2_TeamID','Season'], how = 'left')

tourney_results = tourney_results.loc[tourney_results['DayNum'] >= 136].reset_index(drop=True)
tourney_results = tourney_results.loc[tourney_results['Season'] < 2015].reset_index(drop=True)

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

seeds_T1 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T2 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed','T1_division']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed','T2_division']

tourney_results = tourney_results.merge(seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_results = tourney_results.merge(seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

tourney_results['T1_powerrank'] = tourney_results.groupby(['Season','T1_division'])['T1_quality'].rank(method='dense', ascending=False).astype('Int64')
tourney_results['T2_powerrank'] = tourney_results.groupby(['Season','T2_division'])['T2_quality'].rank(method='dense', ascending=False).astype('Int64')

tourney_results['win'] = np.where(tourney_results['T1_Score'] > tourney_results['T2_Score'], 1, 0)

mean_win_ratio = pd.DataFrame({'seed_win_ratio': tourney_results.groupby('T1_seed')['win'].mean(),
                               'powerrank_win_ratio': tourney_results.groupby('T1_powerrank')['win'].mean()})
mean_win_ratio = mean_win_ratio.reset_index().rename(columns={'index':'seed'})
mean_win_ratio

,seed,seed_win_ratio,powerrank_win_ratio
0,1,0.823151,NaN
1,2,0.741935,NaN
2,3,0.682243,NaN
3,4,0.638298,NaN
4,5,0.558442,NaN
5,6,0.488722,NaN
6,7,0.472868,NaN
7,8,0.339806,NaN
8,9,0.364486,NaN
9,10,0.260870,NaN


In [95]:
seeds

,Season,Seed,TeamID,seed,division
0,1998,W01,3330,1,W
1,1998,W02,3163,2,W
2,1998,W03,3112,3,W
3,1998,W04,3301,4,W
4,1998,W05,3272,5,W
...,...,...,...,...,...
1467,2021,Z12,3141,12,Z
1468,2021,Z13,3226,13,Z
1469,2021,Z14,3292,14,Z
1470,2021,Z15,3238,15,Z


In [96]:
# シード情報とチームクオリティ、シード勝率の合体
TourneyCompactResults = pd.read_csv(DATA_DIR+'/WNCAATourneyCompactResults.csv')

A_t1 = TourneyCompactResults[TourneyCompactResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "TeamID1"})

A_t2 = TourneyCompactResults[TourneyCompactResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "TeamID2"})
A_t1.index = A_t1.index.rename(['Season', 'TeamID'])
A_t2.index = A_t2.index.rename(['Season', 'TeamID'])
A_t2
A = A_t1.join(A_t2, how='outer').reset_index().drop(['TeamID1', 'TeamID2'], axis=1)
del A_t1, A_t2

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

l_seed = A.merge(seeds, on=['Season', 'TeamID'], how='left').drop('Seed', axis=1)
# l_seed = l_seed.drop('location_N', axis=1) # locationがリークさせていた→シンプルにそのトーナメントでの試合数になってしまうから
# del A, seeds
# l_seed['division_rank'] = l_seed.division.replace({'W':4, 'X':3, 'Y':2, 'Z':1})
# l_seed['seed_ratio'] = l_seed['seed'] * l_seed['division_rank']
# l_seed.drop(['seed', 'division', 'division_rank'], inplace=True, axis=1)
q_seed = l_seed.merge(team_quality, on=['Season', 'TeamID'], how='left').drop(['beta'], axis=1)\
        .merge(mean_win_ratio, on='seed', how='left')
q_seed 

,Season,TeamID,seed,division,quality,seed_win_ratio,powerrank_win_ratio
0,1998,3304,9,W,NaN,0.364486,NaN
1,1998,3307,8,W,NaN,0.339806,NaN
2,1999,3104,5,Y,NaN,0.558442,NaN
3,1999,3112,6,Y,NaN,0.488722,NaN
4,1999,3120,5,W,NaN,0.558442,NaN
...,...,...,...,...,...,...,...
851,2019,3390,2,X,231.592580,0.741935,NaN
852,2019,3397,11,W,15.483497,0.320000,NaN
853,2019,3400,7,Z,37.216571,0.472868,NaN
854,2019,3416,12,Z,24.895487,0.180723,NaN


In [97]:
# シード情報とチームクオリティ、シード勝率の合体
MRegularSeasonDetailedResults = pd.read_csv(DATA_DIR+'/WRegularSeasonDetailedResults.csv')

A_t11 = MRegularSeasonDetailedResults[MRegularSeasonDetailedResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "TeamID1"})

A_t21 = MRegularSeasonDetailedResults[MRegularSeasonDetailedResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "TeamID2"})
A_t11.index = A_t11.index.rename(['Season', 'TeamID'])
A_t21.index = A_t21.index.rename(['Season', 'TeamID'])
A_t21
A1 = A_t11.join(A_t21, how='outer').reset_index().drop(['TeamID1', 'TeamID2'], axis=1)
del A_t11, A_t21

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

l_seed1 = A1.merge(seeds, on=['Season', 'TeamID'], how='left').drop('Seed', axis=1)
# l_seed = l_seed.drop('location_N', axis=1) # locationがリークさせていた→シンプルにそのトーナメントでの試合数になってしまうから
# del A, seeds
# l_seed['division_rank'] = l_seed.division.replace({'W':4, 'X':3, 'Y':2, 'Z':1})
# l_seed['seed_ratio'] = l_seed['seed'] * l_seed['division_rank']
# l_seed.drop(['seed', 'division', 'division_rank'], inplace=True, axis=1)
q_seed1 = l_seed1.merge(team_quality, on=['Season', 'TeamID'], how='left').drop(['beta'], axis=1)\
        .merge(mean_win_ratio, on='seed', how='left')


# シード情報以外で回す場合はここから！

In [453]:
MRSCResults = pd.read_csv(DATA_DIR + '/WRegularSeasonCompactResults.csv')

A_w = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_A"})
N_w = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_N"})
H_w = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_H"})
win = A_w.join(N_w, how='outer').join(H_w, how='outer').fillna(0)

H_l = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_H"})
N_l = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_N"})
A_l = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_A"})
lost = A_l.join(N_l, how='outer').join(H_l, how='outer').fillna(0)

win.index = win.index.rename(['Season', 'TeamID'])
lost.index = lost.index.rename(['Season', 'TeamID'])
wl = win.join(lost, how='outer').reset_index()
wl['win_pct_A'] = wl['win_A'] / (wl['win_A'] + wl['lost_A'])
wl['win_pct_N'] = wl['win_N'] / (wl['win_N'] + wl['lost_N'])
wl['win_pct_H'] = wl['win_H'] / (wl['win_H'] + wl['lost_H'])
wl['win_pct_All'] = (wl['win_A'] + wl['win_N'] + wl['win_H']) / \
    (wl['win_A'] + wl['win_N'] + wl['win_H'] + wl['lost_A']\
     + wl['lost_N'] + wl['lost_H'])

del A_w, N_w, H_w, H_l, N_l, A_l, win, lost

In [454]:
MRSCResults['relScore'] = MRSCResults.WScore - MRSCResults.LScore

w_scr = MRSCResults.loc[:, ['Season', 'WTeamID', 'WScore', 'WLoc','relScore']]
w_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
l_scr = MRSCResults.loc[:, ['Season', 'LTeamID', 'LScore', 'WLoc','relScore']]
l_scr['WLoc'] = l_scr.WLoc.apply(lambda x: 'H' if x == 'A' else 'A' \
                                 if x == 'H' else 'N')
l_scr['relScore'] = -1 * l_scr.relScore 
l_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
wl_scr = pd.concat([w_scr,l_scr])

A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_A", "relScore": "relScore_A"})
N_scr = wl_scr[wl_scr.Loc == 'N'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_N", "relScore": "relScore_N"})
H_scr = wl_scr[wl_scr.Loc == 'H'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_H", "relScore": "relScore_H"})
All_scr = wl_scr.groupby(['Season','TeamID'])['Score','relScore']\
    .mean().rename(columns={"Score": "Score_All", "relScore": "relScore_All"})
scr = A_scr.join(N_scr, how='outer').join(H_scr, how='outer')\
    .join(All_scr, how='outer').fillna(0).reset_index()

del w_scr, l_scr, wl_scr, A_scr, H_scr, N_scr, All_scr

<ipython-input-454-3cd860bc9e4b>:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
<ipython-input-454-3cd860bc9e4b>:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  N_scr = wl_scr[wl_scr.Loc == 'N'].groupby(['Season','TeamID'])\
<ipython-input-454-3cd860bc9e4b>:18: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  H_scr = wl_scr[wl_scr.Loc == 'H'].groupby(['Season','TeamID'])\
<ipython-input-454-3cd860bc9e4b>:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  All_scr = wl_scr.groupby(['Season','TeamID'])['Score','relScore']\


In [455]:
scr

,Season,TeamID,Score_A,relScore_A,Score_N,relScore_N,Score_H,relScore_H,Score_All,relScore_All
0,1998,3102,54.818182,-25.363636,0.0,0.00,59.384615,-16.615385,57.291667,-20.625000
1,1998,3103,66.857143,-11.928571,0.0,0.00,71.466667,-0.200000,69.241379,-5.862069
2,1998,3104,73.937500,6.187500,0.0,0.00,79.571429,21.714286,76.566667,13.433333
3,1998,3106,57.000000,-12.909091,0.0,0.00,65.900000,-2.500000,61.238095,-7.952381
4,1998,3108,65.642857,-5.428571,0.0,0.00,71.222222,11.777778,67.826087,1.304348
...,...,...,...,...,...,...,...,...,...,...
8044,2021,3467,62.285714,-4.142857,0.0,0.00,66.750000,-1.375000,64.666667,-2.666667
8045,2021,3468,65.545455,-10.454545,59.0,-29.00,60.100000,-17.300000,62.772727,-14.409091
8046,2021,3469,44.000000,-36.000000,0.0,0.00,53.000000,-10.000000,48.500000,-23.000000
8047,2021,3470,59.181818,-9.000000,55.5,-8.75,59.500000,-7.166667,58.571429,-8.428571


In [456]:
wl

,Season,TeamID,win_A,win_N,win_H,lost_A,lost_N,lost_H,win_pct_A,win_pct_N,win_pct_H,win_pct_All
0,1998,3102,1.0,0.0,3.0,10.0,0.0,10.0,0.090909,NaN,0.230769,0.166667
1,1998,3103,3.0,0.0,8.0,11.0,0.0,7.0,0.214286,NaN,0.533333,0.379310
2,1998,3104,8.0,0.0,13.0,8.0,0.0,1.0,0.500000,NaN,0.928571,0.700000
3,1998,3106,1.0,0.0,5.0,10.0,0.0,5.0,0.090909,NaN,0.500000,0.285714
4,1998,3108,5.0,0.0,7.0,9.0,0.0,2.0,0.357143,NaN,0.777778,0.521739
...,...,...,...,...,...,...,...,...,...,...,...,...
8044,2021,3467,2.0,0.0,3.0,5.0,0.0,5.0,0.285714,NaN,0.375000,0.333333
8045,2021,3468,3.0,0.0,2.0,8.0,1.0,8.0,0.272727,0.00,0.200000,0.227273
8046,2021,3469,NaN,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN,NaN
8047,2021,3470,3.0,1.0,1.0,8.0,3.0,5.0,0.272727,0.25,0.166667,0.238095


In [457]:
# ここでチーム毎の試合データをもとに特徴量作成をやっている(1試合毎のやつ)
# 攻撃回数=2P試投数 + 3P試投数 + ターンオーバー数 + FT試投数 × 0.43)を用いて算出した。
# そして、総得点数を攻撃回数で除することで攻撃効率(Points Per Possession)を求めた。

MRSDetailedResults = pd.read_csv(DATA_DIR + '/WRegularSeasonDetailedResults.csv')
# 対戦相手のスコアを保持
MRSDetailedResults['WO_PTS'] = MRSDetailedResults.LScore
MRSDetailedResults['LO_PTS'] = MRSDetailedResults.WScore
# 対戦相手のオフェンスリバウンド回数、ディフェンスリバウンド率を保持
MRSDetailedResults['WO_DREB'] = MRSDetailedResults.LDR
MRSDetailedResults['LO_DREB'] = MRSDetailedResults.WDR
MRSDetailedResults['WO_OREB'] = MRSDetailedResults.LOR
MRSDetailedResults['LO_OREB'] = MRSDetailedResults.WOR
MRSDetailedResults['WO_REB'] = MRSDetailedResults.LDR + MRSDetailedResults.LOR
MRSDetailedResults['LO_REB'] = MRSDetailedResults.WDR + MRSDetailedResults.WOR
#対戦相手のFGA
MRSDetailedResults['WO_FGA'] = MRSDetailedResults.LFGA
MRSDetailedResults['LO_FGA'] = MRSDetailedResults.WFGA
# 対戦相手のFTA
MRSDetailedResults['WO_FTA'] = MRSDetailedResults.LFTA
MRSDetailedResults['LO_FTA'] = MRSDetailedResults.WFTA
# 対戦相手のTO
MRSDetailedResults['WO_TO'] = MRSDetailedResults.LTO
MRSDetailedResults['LO_TO'] = MRSDetailedResults.WTO

w = MRSDetailedResults.loc[:, ['Season', 'WScore', 'WTeamID', 'WFGM','WFGA','WFGM3'
                               ,'WFGA3','WFTM','WFTA','WOR','WDR','WAst',
                               'WTO','WStl','WBlk','WPF', 'WO_PTS', 'WO_DREB', 'WO_OREB', 'WO_REB',
                              'WO_FGA', 'WO_FTA', 'WO_TO']]
w.columns = ['Season', 'Score', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF', 'O_PTS', 'O_DREB', 'O_OREB', 'O_REB', 'O_FGA', 'O_FTA', 'O_TO']
l = MRSDetailedResults.loc[:, ['Season', 'LScore', 'LTeamID', 'LFGM','LFGA','LFGM3',
                               'LFGA3','LFTM','LFTA','LOR','LDR','LAst',
                               'LTO','LStl','LBlk','LPF', 'LO_PTS', 'LO_DREB', 'LO_OREB', 'LO_REB',
                              'LO_FGA', 'LO_FTA', 'LO_TO']]
l.columns = ['Season', 'Score', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF', 'O_PTS', 'O_DREB', 'O_OREB', 'O_REB', 'O_FGA', 'O_FTA', 'O_TO']

detail = pd.concat([w,l])
detail['goal_rate'] = detail.FGM / detail.FGA 
detail['3p_goal_rate'] = detail.FGM3 / detail.FGA3  
detail['ft_goal_rate'] = detail.FTM  / detail.FTA 
detail['total_attack'] = detail.FGA + detail.TO + detail.FTA * 0.43
detail['Points_Per_Possession'] = detail.Score / detail.total_attack
# https://bstatsplus.com/glossary このサイト参考にスタッツを計算
# 参考サイトその2:https://www.nogawanogawa.com/entry/basketball_analysis
# POSS:ポゼッション。攻撃回数。
detail['POSS'] = detail.FGA + 0.44 * detail.FTA + detail.TO - detail.OR
# ORTG:オフェンスレーティング。攻撃力。100POSSあたりの得点。
detail['ORTG'] = detail.Score / detail.POSS * 100
# DRTG:ディフェンスレーティング。守備力。100POSSあたりの失点。
detail['DRTG'] = detail.O_PTS / detail.POSS * 100
# NRTG:ネットレーティング。総合力。攻撃力と守備力の差でみる真の力。
detail['NRTG'] = detail.ORTG - detail.DRTG
# eFG%:得点効率。Effective FG%。「Four Factors」の1つ。3Pの価値は2Pの1.5倍という重みを付けて効率を評価。50%を超えるのが目安。
detail['eFG%'] = (detail.FGM + 0.5 * detail.FGM3) / detail.FGA
# PPT:シュート1本あたりの得点。得点期待値？Points Per Shot
detail['FGM2'] = detail.FGM - detail.FGM3 - detail.FTM
detail['PPS'] = (2 * detail.FGM2 + 3 * detail.FGM3) / detail.FGA
# 2PPS:2Pの得点期待値。1を超えるのが目安。
detail['FGA2'] = detail.FGA - detail.FGA3 - detail.FTA
detail['2PPS'] = 2 * detail.FGA2
# 3PPS:3Pの得点期待値。1を超えるのが目安。
detail['3PPS'] = 3 * detail['3p_goal_rate']
# TOV%:ターンオーバー率。「Four Factors」の1つ。1回の攻撃でどれだけTOVするか。少ないほうがよい。
detail['TOV%'] = detail.TO / detail.POSS
# OREB%:オフェンスリバウンド率。「Four Factors」の1つ。OREBの機会にどれだけ奪えるか。OREB機会は自チームのOREBと相手のDREBの合計。
detail['OREB%'] = detail.OR / (detail.OR + detail.O_DREB)
# DREB%:ディフェンスリバウンド率。DREBの機会にどれだけ奪えるか。DREB機会は自チームのDREBと相手のOREBの合計。
detail['DREB%'] = detail.DR / (detail.DR + detail.O_OREB)
# REB%:リバウンド率。リバウンド機会にどれだけ奪えるか。
detail['REB'] = detail.DR + detail.OR
# REB = detail.DR + detail.OR
detail['REB%'] = (detail.REB) / (detail.REB + detail.O_REB)
# FTR:フリースローレーティング。「Four Factors」の1つ。フリースローをどれだけ獲得できるか。アタックできてるかどうかの指標。
detail['FTR'] = detail.FTA / detail.FGA
# TS%:True Shooting Percentage。真のシュート成功率。eFG%がフィールドゴールだけなのに対して、TS%はFTも加味した得点効率。
detail['TS%'] = 0.5 * detail.Score / (detail.FGA + 0.44 * detail.FTA)
# 3P/2P:3PAと2PAの比率。アウトサイドとインサイドのどちらに重きを置いているかを測る指標。
detail['3P/2P'] = detail.FGA3 / detail.FGA2
# AST%:アシスト率。アシストによるフィールドゴールがどれだけあるかを測る指標。
detail['AST%'] = detail.Ast / detail.FGM
# AST/TO:ASTとTOVの比率。アシストが多くターンオーバーが少ないのが優秀とみることが多い。
detail['AST/TO'] = detail.Ast / detail.TO
# PACE:ペース。試合展開の速さの目安。1試合＝40分あたりの攻撃回数。
detail['PACE'] = detail.POSS * 5 / 200
# 以下小川さんの作った特徴量
#対戦相手のPOSS
detail['O_POSS'] = detail.O_FGA + 0.44 * detail.O_FTA + detail.O_TO - detail.O_OREB
#LO_DRTG:対戦相手のディフェンスレーティング
detail['O_DRTG'] = detail.Score / detail.O_POSS * 100
#AdjEM 	自チームのオフェンス　ー　対戦相手のディフェンス
detail['AdjEM'] = detail.ORTG - detail.O_DRTG

# dt = detail.groupby(['Season','TeamID'])['eFG%','TOV%','DREB%','OREB%','FTR','AdjEM'].mean().fillna(0).reset_index()
# dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
#                                          'OR','DR','Ast','TO','Stl','Blk','PF',
#                                           'goal_rate', '3p_goal_rate',
#                                          'ft_goal_rate', 'total_attack', 'Points_Per_Possession',
#                                         'POSS', 'ORTG', 'DRTG', 'NRTG', 'eFG%', 'FGM2', 'PPS', 'FGA2',
#                                         '2PPS', '3PPS', 'TOV%', 'OREB%', 'DREB%', 'FTR', 'TS%',
#                                         'AST%', 'AST/TO', 'PACE', 'AdjEM', 'REB', 'REB%']\
#                                         .mean().fillna(0).reset_index()

dt = detail.groupby(['Season','TeamID'])['AdjEM',
                                         'eFG%', 'TOV%', 'OREB%', 'DREB%', 'FTR']\
                                        .mean().fillna(0).reset_index()

# dt.drop(['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF'], axis=1, inplace=True)
# dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
#                                          'OR','DR','Ast','TO','Stl','Blk','PF',
#                                           'goal_rate', '3p_goal_rate',
#                                          'ft_goal_rate', 'total_attack', 'Points_Per_Possession',
#                                         'POSS', 'ORTG', 'DRTG', 'NRTG', 'eFG%', 'FGM2', 'PPS', 'FGA2',
#                                         'TOV%', 'OREB%', 'DREB%', 'FTR', 'TS%',
#                                         'AST%', 'AST/TO', 'PACE', '2PPS', '3PPS']\
#                                         .mean().fillna(0).reset_index()

del w, l

<ipython-input-457-a351cd53257c>:105: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dt = detail.groupby(['Season','TeamID'])['AdjEM',


In [458]:
dt

,Season,TeamID,AdjEM,eFG%,TOV%,OREB%,DREB%,FTR
0,2010,3102,0.400898,0.407876,0.272474,0.345366,0.608409,0.257720
1,2010,3103,0.603569,0.441526,0.269755,0.391232,0.634645,0.364156
2,2010,3104,-0.064566,0.436584,0.257695,0.352448,0.636028,0.273776
3,2010,3105,0.530341,0.437404,0.342812,0.384361,0.609626,0.505554
4,2010,3106,-0.419024,0.377771,0.290722,0.389308,0.671988,0.465138
...,...,...,...,...,...,...,...,...
4158,2021,3467,-0.930487,0.461032,0.233820,0.295018,0.683951,0.242823
4159,2021,3468,-1.212239,0.407715,0.209055,0.264196,0.672499,0.309169
4160,2021,3469,0.829515,0.348813,0.247231,0.232456,0.709244,0.292941
4161,2021,3470,0.756750,0.389943,0.200329,0.227059,0.697541,0.449960


In [459]:
#get seed and team data for elo ratings
df_seed = pd.read_csv(DATA_DIR+'/WNCAATourneySeeds.csv')
df_teams = pd.read_csv(DATA_DIR+'/WTeams.csv')
df_teams['tm_join'] = df_teams.TeamName.apply(lambda x: x.replace('St','State'))

#get 2021 data & join seed w/ team names
df_s_2021 = df_seed[df_seed['Season'] == 2021]
seed_tms = pd.merge(df_s_2021, df_teams.loc[:,['TeamID','TeamName','tm_join']], on='TeamID')

In [460]:
#get elo data from warrennolan.com - Great resource (check it out!)
df_elo = pd.read_html('http://warrennolan.com/basketballw/2021/elochess')[0]

df_elo_final = df_elo.loc[:,['Team','ELO']]
df_elo_final.head()

,Team,ELO
0,Stanford,1863.35
1,Connecticut,1846.41
2,Baylor,1840.78
3,Maryland,1821.86
4,North Carolina State,1810.09


In [461]:
#join elo data with team and seed data | fill in null values
seed_elos = pd.merge(seed_tms,df_elo_final, left_on = 'tm_join',right_on = 'Team', how='left')

seed_elos.iloc[8,6] = 1597.16
seed_elos.iloc[13,6] = 1519.06
seed_elos.iloc[14,6] = 1481.20
seed_elos.iloc[16,6] = 1278.92
seed_elos.iloc[17,6] = 1395.14
seed_elos.iloc[32,6] = 1469.18

seed_elos.isnull().any()

Season      False
Seed        False
TeamID      False
TeamName    False
tm_join     False
Team         True
ELO          True
dtype: bool

In [462]:
#simple formula to get win probability from elo rating differential
def win_prob_t1(team1_elo,team2_elo):
    elo_diff_m = (team2_elo-team1_elo)/400
    t1_win_prob = 1/(1+10**elo_diff_m)
    return t1_win_prob


In [463]:
# build team class for simulation
class Team:
    def __init__(self, teamid, data, season):
        self.teamid = teamid
        self.data = data[(data['TeamID'] == self.teamid)&
                         (data['Season'] == season)].copy()
        self.team_name = self.data['TeamName'].unique()[0]
    
    def getPointsScored(self):
        return self.data['PtScored'].values

    def getPointsAllowed(self):
        return self.data['PtAllowed'].values
    
    def getAttributes(self):
        self.attributes = dict()
        for col in self.data.columns:
            self.attributes[col] = self.data[col].values
        return self.attributes
    

In [464]:
# make simulation functions

import random as rd

def sim_once(team1,team2):
    score_team1= rd.gauss(team1.getPointsScored().mean(),team1.getPointsScored().std())
    score_team2= rd.gauss(team2.getPointsScored().mean(),team2.getPointsScored().std())
    score_against_team1= rd.gauss(team1.getPointsAllowed().mean(),team1.getPointsAllowed().std())
    score_against_team2= rd.gauss(team2.getPointsAllowed().mean(),team2.getPointsAllowed().std())
    final_score_t1 = (score_team1+score_against_team2)/2
    final_score_t2 = (score_team2+score_against_team1)/2
    if final_score_t1 == final_score_t2:
        sim_once(team1,team2)
    return (final_score_t1,final_score_t2, final_score_t1 > final_score_t2)
    

def sim_multiple(team1,team2,n=100):
    """Takes two teams in and returns win % of t1, t1 point dist, t2 point dist, win loss binary"""
    t1_points = []
    t2_points = []
    w_l = []
    for i in range(n):
        sim = sim_once(team1,team2)
        t1_points.append(sim[0])
        t2_points.append(sim[1])
        w_l.append(sim[2])
    return (sum(w_l)/n, t1_points, t2_points, w_l)

In [465]:
#get data by team 
df = pd.read_csv(DATA_DIR+'/WNCAATourneyDetailedResults.csv')


In [466]:
#Only columns we need 
df.head()
df_winners = df.loc[:,['Season','DayNum','WTeamID','WScore','LScore']]
df_losers = df.loc[:,['Season','DayNum','LTeamID','LScore','WScore']]

#rename columns
df_winners.columns = ['Season','DayNum','TeamID','PtScored','PtAllowed']
df_losers.columns = ['Season','DayNum','TeamID','PtScored','PtAllowed']

df_tm = pd.concat([df_winners,df_losers])

In [467]:


# get team names 
df_tm_names = pd.merge(df_tm,df_teams.loc[:,['TeamID','TeamName']], on ='TeamID')
df_tm_names



,Season,DayNum,TeamID,PtScored,PtAllowed,TeamName
0,2010,138,3124,69,55,Baylor
1,2010,140,3124,49,33,Baylor
2,2010,145,3124,77,62,Baylor
3,2010,147,3124,51,48,Baylor
4,2011,139,3124,66,30,Baylor
...,...,...,...,...,...,...
1255,2019,137,3406,61,110,Towson
1256,2019,137,3349,54,58,Rice
1257,2019,138,3101,38,95,Abilene Chr
1258,2019,138,3347,51,73,Radford


In [468]:
df_tm

,Season,DayNum,TeamID,PtScored,PtAllowed
0,2010,138,3124,69,55
1,2010,138,3173,67,66
2,2010,138,3181,72,37
3,2010,138,3199,75,61
4,2010,138,3207,62,42
...,...,...,...,...,...
625,2019,147,3234,53,85
626,2019,147,3390,68,84
627,2019,151,3332,67,72
628,2019,151,3163,76,81


In [469]:
wl_1 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_1.columns ]

wl_2 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_2.columns ]

scr_1 = scr.copy()
scr_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_1.columns ]

scr_2 = scr.copy()
scr_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_2.columns ]

dt_1 = dt.copy()
dt_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_1.columns ]

dt_2 = dt.copy()
dt_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_2.columns ]

# MOR_1 = MOR.copy()
# MOR_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
#                  else str(col) for col in MOR_1.columns ]

# MOR_2 = MOR.copy()
# MOR_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
#                  else str(col) for col in MOR_2.columns ]

q_seed_1 = q_seed.copy()
q_seed_1.columns = [str(col) + '_1' if col not in ['Season', 'TeamID'] \
                   else str(col) for col in q_seed_1.columns]

q_seed_2 = q_seed.copy()
q_seed_2.columns = [str(col) + '_2' if col not in ['Season', 'TeamID'] \
                   else str(col) for col in q_seed_2.columns]

MOR_1 = df_tm.copy()
MOR_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_1.columns ]

MOR_2 = df_tm.copy()
MOR_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_2.columns ]                 

In [470]:
TCResults = pd.read_csv(DATA_DIR + '/WNCAATourneyCompactResults.csv')

tourney1 = TCResults.loc[:, ['Season','WTeamID','LTeamID']]
tourney1.columns = ['Season','TeamID1','TeamID2']
tourney1['result'] = 1

tourney2 = TCResults.loc[:, ['Season','LTeamID','WTeamID']]
tourney2.columns = ['Season','TeamID1','TeamID2']
tourney2['result'] = 0

tourney = pd.concat([tourney1, tourney2])
del tourney1, tourney2

In [471]:
regular_results['T1_TeamID'] = regular_results['T1_TeamID'].astype(np.int64)
regular_results['T2_TeamID'] = regular_results['T2_TeamID'].astype(np.int64)
tourney1 = regular_results.loc[:, ['Season','T1_TeamID','T2_TeamID', 'T1_win']]
tourney1.columns = ['Season','TeamID1','TeamID2', 'result']

tourney2 = regular_results.loc[:, ['Season','T2_TeamID','T1_TeamID', 'T2_win']]
tourney2.columns = ['Season','TeamID1','TeamID2', 'result']

regular_final = pd.concat([tourney1, tourney2])
del tourney1, tourney2
regular_final

,Season,TeamID1,TeamID2,result
0,1998,3104,3202,1
1,1998,3202,3104,0
2,1998,3163,3221,1
3,1998,3221,3163,0
4,1998,3222,3261,1
...,...,...,...,...
231473,2021,3251,3195,0
231474,2021,3372,3358,1
231475,2021,3358,3372,0
231476,2021,3179,3133,0


In [472]:
wl_1

,Season,TeamID,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1
0,1998,3102,0.090909,NaN,0.230769,0.166667
1,1998,3103,0.214286,NaN,0.533333,0.379310
2,1998,3104,0.500000,NaN,0.928571,0.700000
3,1998,3106,0.090909,NaN,0.500000,0.285714
4,1998,3108,0.357143,NaN,0.777778,0.521739
...,...,...,...,...,...,...
8044,2021,3467,0.285714,NaN,0.375000,0.333333
8045,2021,3468,0.272727,0.00,0.200000,0.227273
8046,2021,3469,NaN,NaN,NaN,NaN
8047,2021,3470,0.272727,0.25,0.166667,0.238095


In [473]:
MOR_2

,Season,DayNum_2,TeamID,PtScored_2,PtAllowed_2
0,2010,138,3124,69,55
1,2010,138,3173,67,66
2,2010,138,3181,72,37
3,2010,138,3199,75,61
4,2010,138,3207,62,42
...,...,...,...,...,...
625,2019,147,3234,53,85
626,2019,147,3390,68,84
627,2019,151,3332,67,72
628,2019,151,3163,76,81


In [474]:
def merge_data(df):

    df = df.merge(wl_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(wl_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)


    df = df.merge(scr_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(scr_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df['win_pct_A_diff'] = df['win_pct_A_1'] - df['win_pct_A_2']
    df['win_pct_N_diff'] = df['win_pct_N_1'] - df['win_pct_N_2']
    df['win_pct_H_diff'] = df['win_pct_H_1'] - df['win_pct_H_2']
    df['win_pct_All_diff'] = df['win_pct_All_1'] - df['win_pct_All_2']

    df['Score_A_diff'] = df['Score_A_1'] - df['Score_A_2']
    df['Score_N_diff'] = df['Score_N_1'] - df['Score_N_2']
    df['Score_H_diff'] = df['Score_H_1'] - df['Score_H_2']
    df['Score_All_diff'] = df['Score_All_1'] - df['Score_All_2']

    df['relScore_A_diff'] = df['relScore_A_1'] - df['relScore_A_2']
    df['relScore_N_diff'] = df['relScore_N_1'] - df['relScore_N_2']
    df['relScore_H_diff'] = df['relScore_H_1'] - df['relScore_H_2']
    df['relScore_All_diff'] = df['relScore_All_1'] - df['relScore_All_2']

    df = df.merge(dt_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(dt_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df = df.merge(MOR_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(MOR_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)
    
    df = df.merge(q_seed_1, how='left', left_on=['Season', 'TeamID1'],
                 right_on=['Season', 'TeamID'])
    df = df.merge(q_seed_2, how='left', left_on=['Season', 'TeamID2'],
                 right_on=['Season', 'TeamID'])
    df = df.drop(['TeamID_x', 'TeamID_y'], axis=1)


    df = df.fillna(df.median())
    
    for col in df.columns:
        if (df[col] == np.inf).any() or (df[col] == -np.inf).any():
            df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = df.fillna(df.median())
    
    return df

# tourney = merge_data(tourney)
final_regular_result = merge_data(regular_final)
final_regular_result = final_regular_result.loc[final_regular_result.Season >= 2010,:].reset_index(drop=True)

if STAGE_1:
    final_regular_result = final_regular_result.loc[final_regular_result.Season < 2022, :]

/Users/sasanoshouta/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [475]:
    df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,...,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,...,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,...,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,...,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,...,26,11,17,16,22,9,10,3,4,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2019,147,3124,85,3234,53,N,0,37,70,...,10,14,20,7,19,12,16,11,1,14
626,2019,147,3323,84,3390,68,H,0,33,75,...,22,4,6,14,24,12,12,2,1,12
627,2019,151,3124,72,3332,67,N,0,31,57,...,32,5,7,14,19,14,13,7,1,15
628,2019,151,3323,81,3163,76,N,0,30,77,...,28,6,9,11,26,20,7,4,8,14


In [476]:
final_regular_result=final_regular_result.drop(columns=['division_1','division_2','powerrank_win_ratio_1','powerrank_win_ratio_2','DayNum_1','DayNum_2'])


In [477]:
if STAGE_1:
    MSampleSubmission = pd.read_csv(DATA_DIR + '/WSampleSubmissionStage2.csv')
else:
    MSampleSubmission = pd.read_csv(DATA_DIR + None) # put stage 2 submission file link here

test1 = MSampleSubmission.copy()
test1['Season'] = test1.ID.apply(lambda x: int(x[0:4]))
test1['TeamID1'] = test1.ID.apply(lambda x: int(x[5:9]))
test1['TeamID2'] = test1.ID.apply(lambda x: int(x[10:14]))

test2 = MSampleSubmission.copy()
test2['Season'] = test2.ID.apply(lambda x: int(x[0:4]))
test2['TeamID1'] = test2.ID.apply(lambda x: int(x[10:14]))
test2['TeamID2'] = test2.ID.apply(lambda x: int(x[5:9]))

test = pd.concat([test1,test2]).drop(['Pred'], axis=1)
test = merge_data(test)

/Users/sasanoshouta/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [478]:
final_regular_result.to_csv('final_regular_result_W.csv', index=False)

In [479]:
test=test.drop(columns=['division_1','division_2','powerrank_win_ratio_1','powerrank_win_ratio_2','DayNum_1','DayNum_2'])

In [480]:
test.to_csv('stage1_test_W.csv', index=False)

In [481]:
final_regular_result.columns

Index(['Season', 'TeamID1', 'TeamID2', 'result', 'win_pct_A_1', 'win_pct_N_1',
       'win_pct_H_1', 'win_pct_All_1', 'win_pct_A_2', 'win_pct_N_2',
       'win_pct_H_2', 'win_pct_All_2', 'Score_A_1', 'relScore_A_1',
       'Score_N_1', 'relScore_N_1', 'Score_H_1', 'relScore_H_1', 'Score_All_1',
       'relScore_All_1', 'Score_A_2', 'relScore_A_2', 'Score_N_2',
       'relScore_N_2', 'Score_H_2', 'relScore_H_2', 'Score_All_2',
       'relScore_All_2', 'win_pct_A_diff', 'win_pct_N_diff', 'win_pct_H_diff',
       'win_pct_All_diff', 'Score_A_diff', 'Score_N_diff', 'Score_H_diff',
       'Score_All_diff', 'relScore_A_diff', 'relScore_N_diff',
       'relScore_H_diff', 'relScore_All_diff', 'AdjEM_1', 'eFG%_1', 'TOV%_1',
       'OREB%_1', 'DREB%_1', 'FTR_1', 'AdjEM_2', 'eFG%_2', 'TOV%_2', 'OREB%_2',
       'DREB%_2', 'FTR_2', 'PtScored_1', 'PtAllowed_1', 'PtScored_2',
       'PtAllowed_2', 'seed_1', 'quality_1', 'seed_win_ratio_1', 'seed_2',
       'quality_2', 'seed_win_ratio_2'],
      dty

In [482]:
4556 / len(test.TeamID1.unique())

71.1875

In [483]:
test

,ID,Season,TeamID1,TeamID2,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,...,PtScored_1,PtAllowed_1,PtScored_2,PtAllowed_2,seed_1,quality_1,seed_win_ratio_1,seed_2,quality_2,seed_win_ratio_2
0,2021_3104_3112,2021,3104,3112,0.727273,0.5,0.583333,0.640000,0.571429,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021_3104_3116,2021,3104,3116,0.727273,0.5,0.583333,0.640000,0.500000,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021_3104_3124,2021,3104,3124,0.727273,0.5,0.583333,0.640000,0.909091,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021_3104_3125,2021,3104,3125,0.727273,0.5,0.583333,0.640000,0.818182,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021_3104_3133,2021,3104,3133,0.727273,0.5,0.583333,0.640000,0.454545,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,2021_3450_3460,2021,3460,3450,0.769231,1.0,0.600000,0.720000,0.444444,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4028,2021_3450_3461,2021,3461,3450,0.300000,1.0,0.800000,0.608696,0.444444,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4029,2021_3452_3460,2021,3460,3452,0.769231,1.0,0.600000,0.720000,0.666667,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4030,2021_3452_3461,2021,3461,3452,0.300000,1.0,0.800000,0.608696,0.666667,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
